In [3]:
import pandas as pd
import re

save_path: str = ('../../data/training_data/')

In [4]:
my_runs = pd.read_json("../../data/raw/matchit_my_clembench_all_gameversions.jsonl", lines = True)
repo_runs = pd.read_json("../../data/raw/matchit_ascii_all.jsonl", lines = True)
#fd = pd.read_json("../../data/raw/matchit_multimodal_all.jsonl", lines = True)
df = pd.concat([my_runs, repo_runs])
# since some of the model runs are duplicated
df = df.drop_duplicates(subset=["game", "game_id", "model", "experiment", "episode"])
df.head()

,game,game_id,model,experiment,episode,Aborted,Lose,Success,chat_p1,chat_p2,grid_a,grid_b
0,matchit_ascii,0,Llama-3-70B-Instruct-t0.0--Llama-3-70B-Instruc...,0_same_grid,episode_0,1,0,0,"[{'role': 'user', 'content': 'You are particip...","[{'role': 'user', 'content': 'You are particip...",X X X X X\nX ▢ X ▢ X\nX ▢ X ▢ X\nX ▢ X ▢ X\nX ...,X X X X X\nX ▢ X ▢ X\nX ▢ X ▢ X\nX ▢ X ▢ X\nX ...
1,matchit_ascii,1,Llama-3-70B-Instruct-t0.0--Llama-3-70B-Instruc...,0_same_grid,episode_1,0,0,1,"[{'role': 'user', 'content': 'You are particip...","[{'role': 'user', 'content': 'You are particip...",▢ ▢ X ▢ ▢\n▢ ▢ X ▢ ▢\n▢ ▢ ▢ ▢ ▢\n▢ ▢ X ▢ ▢\n▢ ...,▢ ▢ X ▢ ▢\n▢ ▢ X ▢ ▢\n▢ ▢ ▢ ▢ ▢\n▢ ▢ X ▢ ▢\n▢ ...
2,matchit_ascii,2,Llama-3-70B-Instruct-t0.0--Llama-3-70B-Instruc...,0_same_grid,episode_2,0,0,1,"[{'role': 'user', 'content': 'You are particip...","[{'role': 'user', 'content': 'You are particip...",▢ ▢ ▢ ▢ X\n▢ ▢ ▢ X ▢\n▢ ▢ X ▢ ▢\n▢ X ▢ ▢ ▢\nX ...,▢ ▢ ▢ ▢ X\n▢ ▢ ▢ X ▢\n▢ ▢ X ▢ ▢\n▢ X ▢ ▢ ▢\nX ...
3,matchit_ascii,3,Llama-3-70B-Instruct-t0.0--Llama-3-70B-Instruc...,0_same_grid,episode_3,1,0,0,"[{'role': 'user', 'content': 'You are particip...","[{'role': 'user', 'content': 'You are particip...",▢ ▢ ▢ ▢ ▢\n▢ ▢ ▢ ▢ ▢\nX X ▢ X X\n▢ ▢ ▢ ▢ ▢\n▢ ...,▢ ▢ ▢ ▢ ▢\n▢ ▢ ▢ ▢ ▢\nX X ▢ X X\n▢ ▢ ▢ ▢ ▢\n▢ ...
4,matchit_ascii,4,Llama-3-70B-Instruct-t0.0--Llama-3-70B-Instruc...,0_same_grid,episode_4,0,0,1,"[{'role': 'user', 'content': 'You are particip...","[{'role': 'user', 'content': 'You are particip...",▢ X X X ▢\nX ▢ X ▢ X\nX X ▢ X X\nX ▢ X ▢ X\n▢ ...,▢ X X X ▢\nX ▢ X ▢ X\nX X ▢ X X\nX ▢ X ▢ X\n▢ ...


In [5]:
def contains_specific_text(row, text):
    return any(text.search(d['content']) for d in row)

#verbatim descriptions
verbatim_desc = re.compile('DESCRIPTION:\s?(\\n)?((X|\\u25a2|\s){9}\\n){4}(X|\\u25a2|\s){9}')
#shapes etc.
wrong_domain = re.compile("color|colour|shape|size|orientation|font|crossword|4x4|6x6")
# wrong encoding?
wrong_encoding = re.compile('\\ufffd')

df["verbatim_desc"] = df['chat_p1'].apply(contains_specific_text, text = verbatim_desc)
df["wrong_encoding"] = df['chat_p1'].apply(contains_specific_text, text = wrong_encoding)
df["wrong_domain"] = df['chat_p1'].apply(contains_specific_text, text = wrong_domain)

df.head()

,game,game_id,model,experiment,episode,Aborted,Lose,Success,chat_p1,chat_p2,grid_a,grid_b,verbatim_desc,wrong_encoding,wrong_domain
0,matchit_ascii,0,Llama-3-70B-Instruct-t0.0--Llama-3-70B-Instruc...,0_same_grid,episode_0,1,0,0,"[{'role': 'user', 'content': 'You are particip...","[{'role': 'user', 'content': 'You are particip...",X X X X X\nX ▢ X ▢ X\nX ▢ X ▢ X\nX ▢ X ▢ X\nX ...,X X X X X\nX ▢ X ▢ X\nX ▢ X ▢ X\nX ▢ X ▢ X\nX ...,False,False,True
1,matchit_ascii,1,Llama-3-70B-Instruct-t0.0--Llama-3-70B-Instruc...,0_same_grid,episode_1,0,0,1,"[{'role': 'user', 'content': 'You are particip...","[{'role': 'user', 'content': 'You are particip...",▢ ▢ X ▢ ▢\n▢ ▢ X ▢ ▢\n▢ ▢ ▢ ▢ ▢\n▢ ▢ X ▢ ▢\n▢ ...,▢ ▢ X ▢ ▢\n▢ ▢ X ▢ ▢\n▢ ▢ ▢ ▢ ▢\n▢ ▢ X ▢ ▢\n▢ ...,False,False,False
2,matchit_ascii,2,Llama-3-70B-Instruct-t0.0--Llama-3-70B-Instruc...,0_same_grid,episode_2,0,0,1,"[{'role': 'user', 'content': 'You are particip...","[{'role': 'user', 'content': 'You are particip...",▢ ▢ ▢ ▢ X\n▢ ▢ ▢ X ▢\n▢ ▢ X ▢ ▢\n▢ X ▢ ▢ ▢\nX ...,▢ ▢ ▢ ▢ X\n▢ ▢ ▢ X ▢\n▢ ▢ X ▢ ▢\n▢ X ▢ ▢ ▢\nX ...,False,False,False
3,matchit_ascii,3,Llama-3-70B-Instruct-t0.0--Llama-3-70B-Instruc...,0_same_grid,episode_3,1,0,0,"[{'role': 'user', 'content': 'You are particip...","[{'role': 'user', 'content': 'You are particip...",▢ ▢ ▢ ▢ ▢\n▢ ▢ ▢ ▢ ▢\nX X ▢ X X\n▢ ▢ ▢ ▢ ▢\n▢ ...,▢ ▢ ▢ ▢ ▢\n▢ ▢ ▢ ▢ ▢\nX X ▢ X X\n▢ ▢ ▢ ▢ ▢\n▢ ...,False,False,False
4,matchit_ascii,4,Llama-3-70B-Instruct-t0.0--Llama-3-70B-Instruc...,0_same_grid,episode_4,0,0,1,"[{'role': 'user', 'content': 'You are particip...","[{'role': 'user', 'content': 'You are particip...",▢ X X X ▢\nX ▢ X ▢ X\nX X ▢ X X\nX ▢ X ▢ X\n▢ ...,▢ X X X ▢\nX ▢ X ▢ X\nX X ▢ X X\nX ▢ X ▢ X\n▢ ...,False,False,False


In [6]:
print("# of verbatim grid descriptions:",df.verbatim_desc.sum())
print("# of episodes containing words from wrong domains:", df.wrong_domain.sum())
print("# of episodes containing \\ufffd:", df.wrong_encoding.sum())

print("----- \n# of successful episodes without those specific words:", len(df.query('Success == 1 and verbatim_desc == False and wrong_encoding == False and wrong_domain == False')))


# of verbatim grid descriptions: 775
# of episodes containing words from wrong domains: 598
# of episodes containing \ufffd: 45
----- 
# of successful episodes without those specific words: 807


In [7]:
# split the chats:
def prepare_qa_data(data: pd.DataFrame) -> pd.DataFrame:
    result_data: pd.DataFrame = {key: [] for key in data.columns}

    for i, row in data.iterrows():
        chat = row['chat']
        game = row['game']

        chat_last_index = len(chat)

        for i, item in enumerate(chat):
            if item['role'] == 'assistant':
                for key in data.columns:
                    if key != 'chat':
                        result_data[key].append(row[key])
                    else:
                        if i + 1 < chat_last_index:
                            result_data[key].append(chat[:i + 1])
                        else:
                            result_data[key].append(row[key])
    return result_data

In [11]:
suc = df[df.Success == 1]
suc = suc[~(suc.verbatim_desc | suc.wrong_encoding | suc.wrong_domain)]
print(len(df))
print(len(suc))
chat_p1 = (suc
           .drop(columns=["chat_p2"])
           .rename(columns={"chat_p1": "chat"})
           )
chat_p2 = suc.drop(columns=["chat_p1"]).rename(columns={"chat_p2": "chat"})
chat_all = pd.concat([chat_p1, chat_p2])


chat_p1 = pd.DataFrame(prepare_qa_data(chat_p1))

chat_p2 = pd.DataFrame(prepare_qa_data(chat_p2))

chat_all = pd.DataFrame(prepare_qa_data(chat_all))

2594
807


In [14]:
print(len(chat_all))

12608


In [15]:
chat_all.to_csv(save_path + "DM000.csv")
chat_p1.to_csv(save_path + "DM001.csv")
chat_p2.to_csv(save_path + "DM002.csv")